In [ ]:
# Imports
import IPython.core.display as di
import importlib as ilib
import logging
import matplotlib
import matplotlib.cm as cmx
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

import omega.configuration as c
import omega.core.chain as occ
import omega.core.instrument as oci

%matplotlib inline

In [ ]:
# Disable logging output
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [ ]:
def plot_gradient_df(df, title, cname):
    # Figure
    f, (ax1, ax2) = plt.subplots(2, figsize=(18, 8))
    f.subplots_adjust(top=0.93, bottom=0.01, left=0.2, right=0.99)
    gs = gridspec.GridSpec(2, 1, height_ratios=[1, 50]) 
    # Colors
    cm = plt.get_cmap(cname)
    cNorm  = colors.Normalize(vmin=0, vmax=len(df.columns))
    scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cm)
    # Gradient
    gradient = np.linspace(0, 1, 256)
    gradient = np.vstack((gradient, gradient))
    # Gradient Plot
    ax1 = plt.subplot(gs[0])
    ax1.imshow(gradient, aspect='auto', cmap=plt.get_cmap(cname))
    ax1.get_xaxis().set_visible(False)
    ax1.get_yaxis().set_visible(False)
    gs.update(hspace=0.02)
    # Chart plot
    ax2 = plt.subplot(gs[1])
    for idx, col in enumerate(df.columns):
        ax2.plot(df[col], color=scalarMap.to_rgba(idx))
    # Title
    f.suptitle(title, fontsize=16)

In [ ]:
ilib.reload(occ)
stem = 'LH'
di.display_html('<h3>Analysis</h3>', raw=True)
# Chain
fc = occ.FutureChain(stem, oci.FutureType.Spread)
fc.initialize_contracts(occ.Status.Expired, filter='G', initialize_data=True, days_back=120, extra_days=False)
print('Number of contracts: {}'.format(len(fc.contracts)))
print('Contracts: {}'.format(fc.contracts))
# Data
closes = pd.DataFrame()
volumes = pd.DataFrame()
avols = pd.DataFrame()
for ct in fc.contracts[5:]:
    closes[ct] = fc.data[ct]['Close'].values
    volumes[ct] = fc.data[ct]['Volume'].values
    avols[ct] = occ.aggregate(fc, ct, 'Volume', length=5)

plot_gradient_df(closes, '{} - Closes'.format(stem), 'inferno')
plot_gradient_df(volumes, '{} - Volumes'.format(stem), 'inferno')
plot_gradient_df(avols, '{} - Aggregated Volumes'.format(stem), 'inferno')

In [ ]:
df = pd.read_csv(os.path.join(c.cfg['default']['data'], 'Analysis', 'LC-F.csv'),index_col=0)
df = df.rolling(20, axis=0).mean()
plot_gradient_df(df.transpose(), 'Test', 'inferno')